In [13]:
import random
import matplotlib.pyplot as plt
import numpy as np
from sympy import symbols, simplify
from sympy.physics.paulialgebra import Pauli, evaluate_pauli_product
from sympy.physics.quantum.tensorproduct import TensorProduct, tensor_product_simp

# Define Pauli matrices and identity
sx = Pauli(1)
sy = Pauli(2)
sz = Pauli(3)
s0 = sx * sx
SS = {"I" : s0, "X" : sx, "Y" : sy, "Z" : sz}

# Define stabilizer operators
XXIIIXX = TensorProduct(sx, sx, s0, s0, s0, sx, sx)
IXXXIIX = TensorProduct(s0, sx, sx, sx, s0, s0, sx)
IIIXXXX = TensorProduct(s0, s0, s0, sx, sx, sx, sx)
ZZIIIZZ = TensorProduct(sz, sz, s0, s0, s0, sz, sz)
IZZZIIZ = TensorProduct(s0, sz, sz, sz, s0, s0, sz)
IIIZZZZ = TensorProduct(s0, s0, s0, sz, sz, sz, sz)

# Define stabilizers as lists
stabilizerX = [XXIIIXX, IXXXIIX, IIIXXXX]
stabilizerZ = [ZZIIIZZ, IZZZIIZ, IIIZZZZ]
stabilizerXstr = ["XXIIIXX", "IXXXIIX", "IIIXXXX"]
stabilizerZstr = ["ZZIIIZZ", "IZZZIIZ", "IIIZZZZ"]

def string_to_operator(string):
    """
    Convert a string representation of an operator to its tensor product form.
    """
    operators = [globals()[char] for char in string]
    result = matrix_tensor_product(*operators)
    return result

In [12]:
def multiply_strings(string1, string2):
    """
    Multiply two operator strings by converting them to operator matrices and performing tensor product.
    """
    if string1 == string2:
        return matrix_tensor_product(s0, s0, s0, s0, s0, s0, s0)
    string1_op = string_to_operator(string1)
    string2_op = string_to_operator(string2)
    prod = tensor_product_simp(string1_op * string2_op)
    return prod

def generate_syndrome(error, stabilizers):
    """
    Generate the syndrome for a given error chain and a list of stabilizers.
    """
    error_op = string_to_operator(error)
    syndrome = []
    for stab in stabilizers:
        synd = error_op * stab * error_op * stab
        synd = simplify(synd)
        synd = tensor_product_simp(synd)
        synd = synd.replace(TensorProduct(s0, s0, s0, s0, s0, s0, s0), 1)
        syndrome.append(synd)
    return syndrome


def error_to_corrections(error, stabilizers):
    """
    Determine possible corrections for a given error chain and a list of stabilizers.
    """
    stabilizers_and_Id = stabilizers + [s0 * s0 * s0 * s0 * s0 * s0 * s0]
    list_differences = []
    for stabilizer in stabilizers_and_Id:
        diff = sum(error[i] != stabilizer[i] for i in range(len(error)))
        list_differences.append(diff)
    list_differences = np.array(list_differences)
    min_diff_arg = np.argmin(list_differences)
    stabilizers_and_Id[-1] = error  # correction for Id is the error itself
    return list(np.array(stabilizers_and_Id)[list_differences == list_differences[min_diff_arg]])

# Define the error chain, probability, and syndrome for Z errors with X stabilizers
def Zerror_prob_syndrome(i, p):
    i_binary = "{:0>7}".format(str(bin(i))[2:])
    error_chain = i_binary.replace("0", "I").replace("1", "Z")
    numZ = error_chain.count("Z")
    prob = p**numZ * (1 - p)**(7 - numZ)
    syndrome = generate_syndrome(error_chain, stabilizerX)
    return error_chain, prob, syndrome

p = symbols("p")
syndrome_to_error = {}
pvals = np.arange(0.41, 0.4199, 0.01)
success_probs = []

# Calculate error chains, probabilities, and syndromes for Z errors with X stabilizers
for i in range(128):
    error_chain, prob, syndrome = Zerror_prob_syndrome(i, p)
    if syndrome in syndrome_to_error:
        syndrome_to_error[syndrome][0].append(error_chain)
        syndrome_to_error[syndrome][1].append(prob)
    else:
        syndrome_to_error[syndrome] = [[error_chain], [prob]]

# Calculate success probabilities for each error probability
for pval in pvals:
    syndrome_to_corrections = {}
    for key in syndrome_to_error:
        prob_list = syndrome_to_error[key][1]
        prob_vals = np.array([val.subs(p, pval) for val in prob_list])
        prob_argmax = np.argmax(prob_vals)
        for i, prob in enumerate(prob_list):
            if simplify(prob - prob_list[prob_argmax]) == 0:
                error = syndrome_to_error[key][0][i]
                corrections = error_to_corrections(error, stabilizerZstr)
                try:
                    syndrome_to_corrections[key].extend(corrections)
                except KeyError:
                    syndrome_to_corrections[key] = corrections

    success_prob = 0.0
    for i in range(128):
        error, prob, syndrome = Zerror_prob_syndrome(i, p)
        num_corrections = len(syndrome_to_corrections[syndrome])
        for correction in syndrome_to_corrections[syndrome]:
            success = False
            error_times_correction = multiply_strings(error, correction)
            if error_times_correction == matrix_tensor_product(s0, s0, s0, s0, s0, s0, s0):
                success = True
            elif error_times_correction in stabilizerZ:
                success = True
            if success:
                success_prob += prob.subs(p, pval) * (1.0 / num_corrections)

    success_probs.append(success_prob)

np.savetxt("success_probs.txt", np.c_[pvals, success_probs])


NameError: name 'matrix_tensor_product' is not defined